In [1]:
import os
import analysis_utils as au
import quick_metadata_plots as qmplt
import padeopsIO as pio
import numpy as np

modal_folder = os.path.join(au.DATA_PATH, "F_0013_X_SU_PI_Files")

In [ ]:
img_folder = qmplt.plot_instantaneous_field(modal_folder, 0, tidx = "all", field = "u", zlim = [-5, 5], ylim = 0)
qmplt.film_instantaneous_field(img_folder, fps = 10, video_name = "stationary.mp4")

In [ ]:
img_folder = qmplt.plot_instantaneous_field(modal_folder, 1, tidx = "all", field = "u", zlim = [-5, 5], ylim = 0)

In [34]:
run_folder = au.get_run_folder(modal_folder, 1)
img_folder = os.path.join(run_folder, "u_plots")
sim = pio.BudgetIO(run_folder, padeops = True, runid = 0, normalize_origin="turbine")
sim.slice(field_terms=["u"], tidx = 100)

BudgetIO loaded fields ['u'] at tidx: 100, time: 6.200000


/work2/10264/sgering/stampede3/src/padeopsIO/padeopsIO/budgetIO.py:254: UserWarning: _init_padeops(): Sim_0001 no budget files found.
  warnings.warn(f"_init_padeops(): {self.filename} no budget files found.")


<xarray.GridDataset> Size: 34MB
Dimensions:  (x: 256, y: 128, z: 128)
Coordinates:
  * x        (x) float64 2kB -5.0 -4.902 -4.805 -4.707 ... 19.61 19.71 19.8 19.9
  * y        (y) float64 1kB -5.0 -4.922 -4.844 -4.766 ... 4.766 4.844 4.922
  * z        (z) float64 1kB -2.461 -2.383 -2.305 -2.227 ... 7.305 7.383 7.461
Data variables:
    u        (x, y, z) float64 34MB 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0

In [8]:
qmplt.film_instantaneous_field(img_folder, fps = 50, video_name = "surging.mp4")

MoviePy - Building video /scratch/10264/sgering/Data/F_0013_X_SU_PI_Files/Sim_0001/u_plots/surging.mp4.
MoviePy - Writing video /scratch/10264/sgering/Data/F_0013_X_SU_PI_Files/Sim_0001/u_plots/surging.mp4



MoviePy - Done !
MoviePy - video ready /scratch/10264/sgering/Data/F_0013_X_SU_PI_Files/Sim_0001/u_plots/surging.mp4


In [2]:
img_folder = qmplt.plot_instantaneous_field(modal_folder, 2, tidx = "all", field = "u", zlim = [-5, 5], ylim = 0)
qmplt.film_instantaneous_field(img_folder, fps = 30, video_name = "pitching_velocity.mp4")

/work2/10264/sgering/stampede3/src/padeopsIO/padeopsIO/budgetIO.py:254: UserWarning: _init_padeops(): Sim_0002 no budget files found.
  warnings.warn(f"_init_padeops(): {self.filename} no budget files found.")


BudgetIO loaded fields ['u'] at tidx: 0, time: 0.000000
BudgetIO loaded fields ['u'] at tidx: 10, time: 0.620000
BudgetIO loaded fields ['u'] at tidx: 20, time: 1.240000
BudgetIO loaded fields ['u'] at tidx: 30, time: 1.860000
BudgetIO loaded fields ['u'] at tidx: 40, time: 2.480000
BudgetIO loaded fields ['u'] at tidx: 50, time: 3.100000
BudgetIO loaded fields ['u'] at tidx: 60, time: 3.720000
BudgetIO loaded fields ['u'] at tidx: 70, time: 4.340000
BudgetIO loaded fields ['u'] at tidx: 80, time: 4.960000
BudgetIO loaded fields ['u'] at tidx: 90, time: 5.580000
BudgetIO loaded fields ['u'] at tidx: 100, time: 6.200000
BudgetIO loaded fields ['u'] at tidx: 110, time: 6.820000
BudgetIO loaded fields ['u'] at tidx: 120, time: 7.440000
BudgetIO loaded fields ['u'] at tidx: 130, time: 8.060000
BudgetIO loaded fields ['u'] at tidx: 140, time: 8.680000
BudgetIO loaded fields ['u'] at tidx: 150, time: 9.300000
BudgetIO loaded fields ['u'] at tidx: 160, time: 9.920000
BudgetIO loaded fields ['

MoviePy - Done !
MoviePy - video ready /scratch/10264/sgering/Data/F_0013_X_SU_PI_Files/Sim_0002/u_plots/pitching_velocity.mp4


In [3]:
img_folder = qmplt.plot_instantaneous_field(modal_folder, 2, tidx = "all", field = "p", zlim = [-5, 5], ylim = 0)
qmplt.film_instantaneous_field(img_folder, fps = 30, video_name = "pitching_pressure.mp4")

/work2/10264/sgering/stampede3/src/padeopsIO/padeopsIO/budgetIO.py:254: UserWarning: _init_padeops(): Sim_0002 no budget files found.
  warnings.warn(f"_init_padeops(): {self.filename} no budget files found.")


BudgetIO loaded fields ['p'] at tidx: 0, time: 0.000000
BudgetIO loaded fields ['p'] at tidx: 10, time: 0.620000
BudgetIO loaded fields ['p'] at tidx: 20, time: 1.240000
BudgetIO loaded fields ['p'] at tidx: 30, time: 1.860000
BudgetIO loaded fields ['p'] at tidx: 40, time: 2.480000
BudgetIO loaded fields ['p'] at tidx: 50, time: 3.100000
BudgetIO loaded fields ['p'] at tidx: 60, time: 3.720000
BudgetIO loaded fields ['p'] at tidx: 70, time: 4.340000
BudgetIO loaded fields ['p'] at tidx: 80, time: 4.960000
BudgetIO loaded fields ['p'] at tidx: 90, time: 5.580000
BudgetIO loaded fields ['p'] at tidx: 100, time: 6.200000
BudgetIO loaded fields ['p'] at tidx: 110, time: 6.820000
BudgetIO loaded fields ['p'] at tidx: 120, time: 7.440000
BudgetIO loaded fields ['p'] at tidx: 130, time: 8.060000
BudgetIO loaded fields ['p'] at tidx: 140, time: 8.680000
BudgetIO loaded fields ['p'] at tidx: 150, time: 9.300000
BudgetIO loaded fields ['p'] at tidx: 160, time: 9.920000
BudgetIO loaded fields ['

MoviePy - Done !
MoviePy - video ready /scratch/10264/sgering/Data/F_0013_X_SU_PI_Files/Sim_0002/p_plots/pitching_pressure.mp4
